In [1]:
#Dependencies
import pandas as pd

In [2]:
#Read the csv
raw_data = pd.read_csv("../output_data/imdb_all_movie_data.csv")
print(raw_data["Title"].count())
raw_data.head()

1458


,Title,Type,Release,Runtime,Genre,Nominations,Metascore,imdbRating,imdbVotes,imdbID,Budget,Gross
0,Insidious: The Last Key,Movie,2018-1-5,6180.0,Horror,NaN,49.0,5.7,63102,tt5726086,10000000.0,167885588.0
1,The Strange Ones,Movie,2018-1-5,4860.0,Drama,NaN,57.0,5.1,2741,tt6014904,NaN,28374.0
2,Stratton,Movie,2018-1-5,5700.0,Action,NaN,26.0,4.8,5405,tt3567666,NaN,257212.0
3,Sweet Country,Movie,2018-4-6,6780.0,Adventure,NaN,88.0,6.9,8855,tt6958212,NaN,1849718.0
4,The Commuter,Movie,2018-1-12,6240.0,Action,NaN,56.0,6.3,118954,tt1590193,30000000.0,119942387.0


In [3]:
#Date format
true_dates = raw_data.loc[raw_data["Release"].str.contains("None")==False]
true_dates["Release"] = true_dates["Release"].astype("datetime64")

#Filters
filter_df = true_dates.loc[(true_dates["Release"]>="2018-01-01")&
                           (true_dates["Release"]<"2023-01-01")&
                           (true_dates["Type"]=="Movie")&
                           (true_dates["Runtime"]>=5400)]

C:\Users\vinay\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
#Count the remaining rows
filter_df["Title"].count()

1155

In [5]:
#Read the csvs for each platform
netflix = pd.read_csv("resources/wikipedia_netflix_movies_2018_to_2022.csv")
amazon = pd.read_csv("resources/wikipedia_amazon_movies.csv")
disney = pd.read_csv("resources/wikipedia_disney_plus_original_movies.csv")
hulu = pd.read_csv("resources/wikipedia_hulu_movies.csv")

In [6]:
#Add platform columns
netflix["Platform"] = "netflix"
amazon["Platform"] = "amazon"
disney["Platform"] = "disney"
hulu["Platform"] = "hulu"

In [7]:
#Combine streaming platforms
streaming = pd.concat([netflix,amazon,disney,hulu])

In [8]:
#Merge
merge_df = pd.merge(filter_df,streaming,on="Title",how="left")

In [9]:
#Find duplicate titles
dupes = merge_df.loc[merge_df["Title"].duplicated()]["Title"].unique()

#Find indexes for duplicate titles
dupes_df = merge_df.loc[merge_df["Title"].isin(dupes),:]

#Drop all data for those titles
titles_df = merge_df.drop(dupes_df.index)

In [10]:
#Fill null platform types
titles_df["Platform"].fillna("others",inplace=True)

In [11]:
#Export to csv
titles_df.to_csv("../output_data/clean_data_for_analysis.csv",index=False)